In [2]:
import spacy
from spacy import displacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from scipy.special import softmax
import torch
import numpy as np
from tqdm import tqdm

In [14]:
industry_data = pd.read_csv(r'C:\Users\imran\DataspellProjects\WalidCase\data\processed/industry_dataset_clean_some_deleted.csv', sep='\t')
industry_data.insert(0, 'id', industry_data.index)
startups = pd.read_csv(r"C:\Users\imran\DataspellProjects\WalidCase\data\processed\clustered/4_clusters_nouns_adjectives_only.csv", index_col=0)
startups.dropna(inplace=True)

In [15]:
industry_data

,id,industry,keywords
0,0,Procurement,"purchasing, sourcing, supplier, contract, supp..."
1,1,GreenTech,"renewable energy, solar, wind, biofuel, geothe..."
2,2,Esports,"gaming, competition, professional, tournament,..."
3,3,Quantum Computing,"qubit, superposition, entanglement, quantum al..."
4,4,Manufacturing,"production, assembly, factory, automation, mac..."
...,...,...,...
93,93,Big Data,"data processing, analytics, data mining, stora..."
94,94,Connected Home,"smart home, IoT, home automation, security, en..."
95,95,Network Infrastructure,"internet backbone, data transmission, routers,..."
96,96,Food & Beverage,"food industry, restaurant, food production, fo..."


In [85]:
# Load the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Load the tokenizer and model for precomputing label embeddings
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-mnli")
model = BartForSequenceClassification.from_pretrained("facebook/bart-large-mnli")


In [87]:
industry_labels = industry_data['keywords'].tolist()
#industry_labels = [x.replace(',', '') for x in industry_labels]
#industry_labels = [x for x in industry_labels if x != 'nan']
candidate_labels = list(set(industry_labels))

In [16]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

keywords_to_industry = dict(zip(industry_data['keywords'].str.lower(), industry_data['industry']))

In [102]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

hypotheses = candidate_labels
premises = [startups['cb_description'][0]] * len(hypotheses)



{'carrier services, satellite communication, fixed-line network, mobile virtual network operator, last mile connectivity, telecom towers, telecom billing, submarine cables, backhaul, network management': 'Telecoms',
 'mobile applications, mobile devices, mobile operating systems, mobile content, mobile commerce, location-based services, mobile gaming, mobile security, mobile user experience, mobile data': 'Mobile',
 'real-time communication, chat applications, video conferencing, audio conferencing, web conferencing, email, instant messaging, communication apis, social networking, enterprise communication': 'Communications',
 'content delivery network, network function virtualization, data center, optical networking, network topology, ethernet, vpn, mpls, traffic management, network architecture': 'Network Infrastructure',
 '5g radio access network, millimeter wave, 5g small cells, 5g core network, 5g network slicing, massive mimo, beamforming, 5g network deployment, 5g use cases, 5g t

In [17]:
clusters = [['ultimately', 'quick', 'expensive', 'whilst', 'highly', 'early', 'ethical', 'personal', 'end', 'radically', 'legal', 'clarity'], ['payment', 'agency', 'rental', 'compensation', 'accident', 'cost', 'injury', 'medical', 'insurer', 'insurance'], ['claim', 'approach', 'deal', 'provide', 'attention', 'automate', 'customer', 'process', 'interresolve', 'work', 'solution', 'car', 'new', 'inefficient', 'bypass', 'cut']]

clusters = [' '.join(x) for x in clusters]
#sequence_to_classify = clusters[1] + clusters[2]
sequence_to_classify = startups['cb_description'][0]
candidate_labels = industry_data['keywords'].str.lower().tolist()
candidate_labels = [x for x in candidate_labels if x != 'nan']
candidate_labels = list(set(candidate_labels))


results = []
for i, row in tqdm(startups.head(30).iterrows()):
    res = classifier(row['cb_description'], candidate_labels, multi_class=True)

    industry1 = industry_data[industry_data['keywords'].str.lower() == res['labels'][0]]['industry']
    industry2 = industry_data[industry_data['keywords'].str.lower() == res['labels'][1]]['industry']
    industry3 = industry_data[industry_data['keywords'].str.lower() == res['labels'][2]]['industry']

    score1 = res['scores'][0]
    score2 = res['scores'][1]
    score3 = res['scores'][2]

    result = {
        'sequence': row['cb_description'],
        'industry1': industry1.values[0],
        'industry2': industry2.values[0],
        'industry3': industry3.values[0],
        'score1': score1,
        'score2': score2,
        'score3': score3
    }

    results.append(result)

df = pd.DataFrame(results)




0it [00:00, ?it/s]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
1it [01:11, 71.28s/it]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
2it [02:49, 86.95s/it]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
3it [04:03, 80.99s/it]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
4it [05:17, 78.43s/it]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
5it [06:28, 75.72s/it]The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
6it [07:

In [19]:
df.to_csv(r'C:\Users\imran\DataspellProjects\WalidCase\data\tagged\clustered_with_spacy_engineering/oneshotclass_30_records_nouns_adjectives_industryv1')

In [104]:
industry1 = industry_data[industry_data['keywords'].str.lower() == res['labels'][0]]['industry']
industry2 = industry_data[industry_data['keywords'].str.lower() == res['labels'][1]]['industry']
industry3 = industry_data[industry_data['keywords'].str.lower() == res['labels'][2]]['industry']

score1 = res['scores'][0]
score2 = res['scores'][1]
score3 = res['scores'][2]

df = pd.DataFrame({'sequence': [sequence_to_classify], 'industry1': [industry1.values[0]], 'score1': [score1], 'industry2': [industry2.values[0]], 'score2': [score2], 'industry3': [industry3.values[0]], 'score3': [score3]})





{'sequence': 'interresolve radically new approach deal car accident personal injury claim highly automate process work insurer car rental agency bypass expensive inefficient legal process interresolve ethical solution end customer provide early medical attention clarity claim quick payment compensation whilst ultimately cut car insurance cost',
 'labels': ['insurtech platforms, claims automation, p2p insurance, insurance comparison, microinsurance, usage-based insurance, parametric insurance, digital brokerage, risk analytics, insurtech partnerships',
  'task automation, time tracking, team collaboration, process optimization, productivity tools, remote work, time-saving techniques, performance improvement, personal productivity',
  'querying, olap, drill-down analysis, performance metrics, scorecards, bi tools, self-service bi, data lineage, real-time analytics, report automation',
  'ev charging, charge stations, grid integration, battery technology, power conversion, electric drivet

In [107]:
# find out which keywords are mapped to which industries
industry1 = industry_data[industry_data['keywords'].str.lower() == res['labels'][0]]['industry']
industry2 = industry_data[industry_data['keywords'].str.lower() == res['labels'][1]]['industry']
industry3 = industry_data[industry_data['keywords'].str.lower() == res['labels'][2]]['industry']

#print(f"transformer: bart-large-mnli, industry kws: v1\n\nsequence: {res['sequence']}\n\ntag 1: {industry1['industry']}, kws: {res['labels'][0]} score: {res['scores'][0]},\ntag2: {industry2['industry']} kws 2: {res['labels'][1]}, score: {res['scores'][1]}\ntag3: {industry3['industry']},  kws 3: {res['labels'][2]}, score: {res['scores'][2]}\n\nReal tags: InsurTech, Automotive, Insurance")

print(f"sequence: {res['sequence']}\n\nindustry1: {industry1.values[0]}, score: {res['scores'][0]},\nindustry2: {industry2.values[0]} score: {res['scores'][1]}\nindustry3: {industry3.values[0]}, score: {res['scores'][2]}\n\nReal tags: InsurTech, Automotive, Insurance\n\n"
      f"keyword: {res['labels'][0]}, score: {res['scores'][0]},\nkeyword: {res['labels'][1]} score: {res['scores'][1]}\nkeyword: {res['labels'][2]}, score: {res['scores'][2]}\n\n")

sequence: interresolve radically new approach deal car accident personal injury claim highly automate process work insurer car rental agency bypass expensive inefficient legal process interresolve ethical solution end customer provide early medical attention clarity claim quick payment compensation whilst ultimately cut car insurance cost

industry1: InsurTech, score: 0.5340990424156189,
industry2: Productivity score: 0.2110290825366974
industry3: Business Intelligence, score: 0.17066243290901184

Real tags: InsurTech, Automotive, Insurance

keyword: insurtech platforms, claims automation, p2p insurance, insurance comparison, microinsurance, usage-based insurance, parametric insurance, digital brokerage, risk analytics, insurtech partnerships, score: 0.5340990424156189,
keyword: task automation, time tracking, team collaboration, process optimization, productivity tools, remote work, time-saving techniques, performance improvement, personal productivity score: 0.2110290825366974
keywor

In [37]:
industry1.values[0]

'InsurTech'

In [7]:
sequence_to_classify = startups['cb_description'][0]
candidate_labels_2 = industry_data['keywords'].str.lower().tolist()
candidate_labels_2 = [x for x in candidate_labels_2 if x != 'nan']
candidate_labels_2 = list(set(candidate_labels_2))

res = classifier(sequence_to_classify, candidate_labels_2, multi_class=True)

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [8]:
industry_data

,id,industry,keywords
0,0,Telecoms,"carrier services, satellite communication, fix..."
1,1,Mobile,"mobile applications, mobile devices, mobile op..."
2,2,Communications,"real-time communication, chat applications, vi..."
3,3,Network Infrastructure,"content delivery network, network function vir..."
4,4,5G Networks,"5G radio access network, millimeter wave, 5G s..."
...,...,...,...
113,113,Video/Audio,"media production, video platforms, audio engin..."
114,114,Genomics,"functional genomics, gene expression, epigenom..."
115,115,Longevity,"aging biology, lifespan extension, rejuvenatio..."
116,116,Gut Microbiome,"intestinal microbiota, intestinal ecology, gut..."


In [10]:
# find out which keywords are mapped to which industries
industry1 = industry_data[industry_data['keywords'].str.lower() == res['labels'][0]]
industry2 = industry_data[industry_data['keywords'].str.lower() == res['labels'][1]]
industry3 = industry_data[industry_data['keywords'].str.lower() == res['labels'][2]]

print(industry1['industry'].values[0], industry2['industry'].values[0], industry3['industry'].values[0])

InsurTech Productivity Business Intelligence
